Before you start, make sure that you have the correct kernel. Check if the kernel is "Python 2" (there is a legend in the upper right corner, below the "Logout" button). If it is not Python 2, select Kernel → Change Kernel → Python 2 in the menu bar.
Make sure that you fill any place that says "YOUR CODE HERE" or "YOUR ANSWER HERE", and that you erase the line "raise NotImplementedError()" in the code cells.

# Introduction: 

In this Jupyter notebook, you will play around with using Python to manipulate and explore hyperspectral images. <br>

While a typical RGB image has 3 channels (red, green, and blue), hyperspectral images have **many** channels, one for a different wavelength. In fact, a hyperspectral image lets you analyze an entire light **spectrum** at each pixel.<br>

In this notebook, we've written a bunch of python functions that will help you read, visualize, and explore the images. Take a look at them and familiarize yourself with them in the **Function definitions** section. 

# Import modules/libraries

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pickle as pkl
import os
import imageio
import sys
sys.path.append('../tools/')
from Tools import *
%load_ext autoreload
%autoreload 2
%pylab inline

# Function definitions: 

In [ ]:
def load_PARC_image(file_name):
    # This function reads a .bin file from the PARC hyperspectral camera
    # and loads it into python as a 3-D matrix (numpy array)
    my_img = np.fromfile(file_name, dtype = np.uint16)
    my_img = np.reshape(my_img, (141,488,648))
    return my_img

def get_waves(my_img, resolution = 5, init_nm = 400):
    # given a PARC hyperspectral camera image as input
    # returns the range of wavelengths to which each slice/matrix corresponds. 
    waves = np.arange(init_nm, init_nm+resolution*my_img.shape[0], resolution)
    return waves

def get_nm_ind(waves, nm):
    # Returns the wavelength available in a PARC hyperspectral camera image
    # and its corresponding index closest
    # To a wavelength of interest. 
    # ind_nm, nm = get_nm_ind(waves, nm)
    return [(i,waves[i]) for i in range(len(waves)) if 
            np.abs(waves[i]-nm)==np.min(np.abs(waves-nm))][0]

def display_full_single_channel(my_img, ind_nm, nm, save = False, saveDir = 'image_dir'):
    # This function displays a single channel/wavelength/slice
    # in grayscale
    # ind_nm and nm can be obtained from 
    # ind_nm, nm = get_nm_ind(waves, nm)
    plt.figure(figsize = (10,10))
    single_slice = my_img[ind_nm,:,:]
    plt.imshow(single_slice, cmap = "gray")
    plt.title('Single Channel: ' + str(int(nm))+"nm", fontsize = 24)

    if save: 
        path_to_file = os.path.join(saveDir, 'single_channel_'+str(nm)+'_nm.pkl')
        pkl.dump(single_slice, open(path_to_file, "wb"))
    
    
def draw_box(ax, rlim, clim):
    # This function is used to draw a red box around a region 
    # that we'll zoom into. 
    
    xbox = np.linspace(clim[0], clim[1])
    ybox = np.linspace(rlim[0], rlim[1])
    ax.plot( xbox , rlim[0]*np.ones(len(xbox)),'r' )
    ax.plot( xbox , rlim[1]*np.ones(len(xbox)),'r' )
    ax.plot( clim[0]*np.ones(len(ybox)), ybox ,'r' )
    ax.plot( clim[1]*np.ones(len(ybox)), ybox ,'r' )
    
def display_part_single_channel(my_img, ind_nm, nm, rlim, clim, fix_tics = True):
    # This function displays in grayscale a single channel 
    # corresponding to wavelength nm (and index ind_nm)
    # it zooms in to a region specified by the coordinate range rlim and clim
    
    fig, (ax1,ax2) = plt.subplots(1,2, figsize = (14,8))
    ax1.imshow(my_img[ind_nm,:,:], cmap = "gray")
    draw_box(ax1, rlim, clim)
    ax1.set_xlim(0,my_img.shape[2])
    ax1.set_ylim(my_img.shape[1],0)

    ax2.imshow(my_img[ind_nm,rlim[0]:rlim[1],clim[0]:clim[1]], cmap = "gray")
    if fix_tics:
        a=ax2.get_yticks().tolist()
        new_tics = [ int(yt) for yt in np.linspace(rlim[0], rlim[1], len(a))]
        tkn = ax2.set_yticklabels(new_tics)

        old_tics=ax2.get_xticks().tolist()
        new_tics = [ int(xt) for xt in np.linspace(clim[0], clim[1], len(old_tics))]
        tkn = ax2.set_xticklabels(new_tics)
        
def display_single_channel_centered_at(my_img, ind_nm, nm, rcenter, ccenter, wd=20,fix_tics = True):
    # This function displays in grayscale a single channel 
    # corresponding to wavelength nm (and index ind_nm)
    # it zooms in to a region specified by the pixel with coordinates 
    # rcenter and ccenter. 
    
    fig, (ax1,ax2) = plt.subplots(1,2, figsize = (14,8))
    ax1.imshow(my_img[ind_nm,:,:], cmap = "gray")
    draw_box(ax1, [rcenter-wd, rcenter+wd], [ccenter-wd, ccenter+wd])
    ax1.set_xlim(0,my_img.shape[2])
    ax1.set_ylim(my_img.shape[1],0)
    
    ax2.imshow(my_img[ind_nm,rcenter-wd:rcenter+wd,ccenter-wd:ccenter+wd], cmap = "gray")
    if fix_tics:
        a=ax2.get_yticks().tolist()
        new_tics = [ int(yt) for yt in np.linspace(rcenter-wd, rcenter+wd, len(a))]
        tkn = ax2.set_yticklabels(new_tics)

        old_tics=ax2.get_xticks().tolist()
        new_tics = [ int(xt) for xt in np.linspace(ccenter-wd, ccenter+wd, len(old_tics))]
        tkn = ax2.set_xticklabels(new_tics)
        
def get_max_wavelength(spectrum, waves):
    # This function finds the wavelength with the 
    # highest intensity in a spectrum
    
    ind_max = [i for i in range(len(spectrum)) if spectrum[i] == np.max(spectrum)][0]
    return int(waves[ind_max])
        
def single_pxl_spectrum(my_img, ind_nm, nm, rcenter, ccenter, wd = 40):
    # This function plots the full spectrum for a single pixel
    # with coordinates rcenter, ccenter
    
    display_single_channel_centered_at(my_img, ind_nm, nm, rcenter, ccenter, wd)
    
    plt.figure(figsize = (6,6)) 

    waves = get_waves(my_img)
    spectrum = my_img[:, rcenter, ccenter]
    max_freq = get_max_wavelength(spectrum, waves)
    
    plt.plot(waves, spectrum, lw = 3)
    plt.xlabel('Wavelength (nm)', fontsize = 24)
    plt.ylabel('Intensity (units)', fontsize = 24)

    plt.title('Max. Frequency = ' + str(max_freq) + ' nm', fontsize = 24)
    
def hyperspectral_2_RGB(my_img, waves):
    ### This function takes a hyperspectral image
    ### and "converts" it to an RGB representation
    ### it uses some of the functions in the Tools.py library
    ### written by Teresa Tamayo. 
    my_img_rolled = np.rollaxis(np.rollaxis(my_img,1,0),2,1)
    rgb_image = eye_sensitivity(my_img_rolled,waves,tranformation='linear')
    rgb_image *=1.5
    max_pixel = np.max(rgb_image)
    rgb_image_final = rgb_image/max_pixel**1
    return rgb_image_final

With the last functions complete the following cells.

# Read in Hyperspectral image

In [ ]:
## This is the path of the file
file_name = 'image_dir/LEDs_4.bin'
# YOUR CODE HERE
raise NotImplementedError()

# Get the range of wavelengths for which we have data:

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
print(waves.min(), waves.max())

# Display, in grayscale, a single slice/channel/wavelength of the hyperspectral image:

In [ ]:
nm = 660
# YOUR CODE HERE
raise NotImplementedError()


# Zoom-in on an interesting region to look at in more detail by specifying a range of x and y values:

In [ ]:
rlim = [180,260]
clim = [300,500]
display_part_single_channel(my_img, ind_nm, nm, rlim, clim)


# Zoom in on a region by specifying the central pixel:

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Plot pixel spectrum for the different LED strips:

#### Bottom strip:

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

#### Second strip from the bottom:

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

#### Third strip from the bottom:

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

#### Top LED strip:

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

#### Display and save slices/channels/wavelengths of interest:

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Based on the spectrum, what color do you think are these LED strips? <br>

# And how do we make a "regular" RGB image to display in the computer in "color"?

If you have enough time, your instructors will explain how the following works: 

#### Create a "fake" RGB image from the hyperspectral data, and visualize it. 

In [ ]:
rgb_image_final = hyperspectral_2_RGB(my_img, waves)

## Display the image
plt.imshow(rgb_image_final)  
    

#### Save the RGB image to file, in the directory "image_dir". You'll use it when you run the synesthesia pygame code using the hyperspectral image. 

In [ ]:
save_path_file = os.path.join('image_dir', 'LEDs_4_RGB.png')
imageio.imwrite(save_path_file, rgb_image_final)